In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns

In [2]:
df = pd.read_csv('ppg_churn.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   state   5000 non-null   object 
 1   X02     5000 non-null   int64  
 2   X03     5000 non-null   object 
 3   X04     5000 non-null   object 
 4   X05     5000 non-null   object 
 5   X06     5000 non-null   int64  
 6   X07     5000 non-null   float64
 7   X08     5000 non-null   int64  
 8   X09     5000 non-null   float64
 9   X10     5000 non-null   float64
 10  X11     5000 non-null   int64  
 11  X12     5000 non-null   float64
 12  X13     5000 non-null   float64
 13  X14     5000 non-null   int64  
 14  X15     5000 non-null   float64
 15  X16     5000 non-null   float64
 16  X17     5000 non-null   int64  
 17  X18     5000 non-null   float64
 18  X19     5000 non-null   int64  
 19  churn   5000 non-null   object 
dtypes: float64(8), int64(7), object(5)
memory usage: 781.4+ KB


In [4]:
df['lump_x17'] = np.where( df.X17 > 7, 'Other', df.X17.astype('str'))
df['lump_x19'] = np.where( df.X19 > 3, 'Other', df.X19.astype('str'))
df['X06_cat'] = np.where(df['X06'] > 0, 1, 0)

In [5]:
df['X06_cat'] = df.X06_cat.astype('str')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 23 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   state     5000 non-null   object 
 1   X02       5000 non-null   int64  
 2   X03       5000 non-null   object 
 3   X04       5000 non-null   object 
 4   X05       5000 non-null   object 
 5   X06       5000 non-null   int64  
 6   X07       5000 non-null   float64
 7   X08       5000 non-null   int64  
 8   X09       5000 non-null   float64
 9   X10       5000 non-null   float64
 10  X11       5000 non-null   int64  
 11  X12       5000 non-null   float64
 12  X13       5000 non-null   float64
 13  X14       5000 non-null   int64  
 14  X15       5000 non-null   float64
 15  X16       5000 non-null   float64
 16  X17       5000 non-null   int64  
 17  X18       5000 non-null   float64
 18  X19       5000 non-null   int64  
 19  churn     5000 non-null   object 
 20  lump_x17  5000 non-null   obje

In [7]:
df = df.drop(columns=['X06', 'X17', 'X19'])

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 20 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   state     5000 non-null   object 
 1   X02       5000 non-null   int64  
 2   X03       5000 non-null   object 
 3   X04       5000 non-null   object 
 4   X05       5000 non-null   object 
 5   X07       5000 non-null   float64
 6   X08       5000 non-null   int64  
 7   X09       5000 non-null   float64
 8   X10       5000 non-null   float64
 9   X11       5000 non-null   int64  
 10  X12       5000 non-null   float64
 11  X13       5000 non-null   float64
 12  X14       5000 non-null   int64  
 13  X15       5000 non-null   float64
 14  X16       5000 non-null   float64
 15  X18       5000 non-null   float64
 16  churn     5000 non-null   object 
 17  lump_x17  5000 non-null   object 
 18  lump_x19  5000 non-null   object 
 19  X06_cat   5000 non-null   object 
dtypes: float64(8), int64(4), objec

In [9]:
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import RepeatedStratifiedKFold

from sklearn.linear_model import LogisticRegression

In [10]:
from sklearn.pipeline import Pipeline

In [11]:
from sklearn.model_selection import cross_val_score

In [12]:
xinputs = df.select_dtypes('number').copy()
youtput = df.loc[:, ['churn']].copy()

In [13]:
X_train = xinputs.to_numpy()
y_train = youtput.churn.to_numpy().ravel()

In [14]:
y_train.shape

(5000,)

In [15]:
X_train.shape

(5000, 12)

In [16]:
enet_default = LogisticRegression(penalty='elasticnet', solver='saga', random_state=101, max_iter=10001,
                                  C=1.0, l1_ratio=0.5)

In [17]:
default_enet_wflow = Pipeline( steps=[('std_inputs', StandardScaler()), 
                                      ('enet', enet_default)] )

In [18]:
my_cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=101)

In [19]:
enet_default_cv = cross_val_score(default_enet_wflow, X_train, y_train, cv=my_cv)

In [20]:
enet_default_cv


array([0.863, 0.864, 0.864, 0.859, 0.861, 0.863, 0.86 , 0.865, 0.862,
       0.862, 0.863, 0.86 , 0.86 , 0.862, 0.862])

In [21]:
enet_default_cv.mean()

0.8619999999999999

In [22]:
df.churn.value_counts(normalize=True)

no     0.8586
yes    0.1414
Name: churn, dtype: float64

In [23]:
from sklearn.model_selection import GridSearchCV

In [24]:
enet_to_fit = LogisticRegression(penalty='elasticnet', solver='saga', random_state=101, max_iter=10001)

In [25]:
enet_cv_wflow = Pipeline( steps = [('std_inputs', StandardScaler()), 
                                   ('enet', enet_to_fit)] )

In [26]:
enet_grid = {'enet__C': np.exp(np.linspace(-6, 6, num=11)),
             'enet__l1_ratio': np.linspace(0, 1, num=5)}

In [27]:
enet_search = GridSearchCV(enet_cv_wflow, param_grid=enet_grid, cv=my_cv)

In [28]:
enet_search_results = enet_search.fit(X_train, y_train)


In [30]:
enet_search_results.best_score_

0.8620666666666666

In [31]:
enet_search_results.best_params_

{'enet__C': 1.0, 'enet__l1_ratio': 0.0}

## Case - 2 Treating `X17` as continuous as `X19` and `X06` as categorical

In [32]:
df_2 = pd.read_csv('ppg_churn.csv')

In [33]:
df_2['lump_x19'] = np.where( df_2.X19 > 3, 'Other', df_2.X19.astype('str'))
df_2['X06_cat'] = np.where(df_2['X06'] > 0, 1, 0)

In [34]:
df_2['X06_cat'] = df_2.X06_cat.astype('str')

In [35]:
df_2 = df_2.drop(columns=['X06', 'X19'])

In [36]:
xinputs_2 = df_2.select_dtypes('number').copy()
youtput_2 = df_2.loc[:, ['churn']].copy()

In [37]:
X_train_2 = xinputs_2.to_numpy()
y_train_2 = youtput_2.churn.to_numpy().ravel()

In [38]:
X_train_2.shape

(5000, 13)

In [39]:
y_train_2.shape

(5000,)

In [40]:
enet_default_2 = LogisticRegression(penalty='elasticnet', solver='saga', random_state=101, max_iter=10001,
                                  C=1.0, l1_ratio=0.5)

In [41]:
default_enet_wflow_2 = Pipeline( steps=[('std_inputs', StandardScaler()), 
                                      ('enet', enet_default_2)] )

In [42]:
enet_default_cv_2 = cross_val_score(default_enet_wflow_2, X_train_2, y_train_2, cv=my_cv)

In [43]:
enet_default_cv_2

array([0.862, 0.863, 0.865, 0.858, 0.86 , 0.862, 0.861, 0.864, 0.861,
       0.862, 0.861, 0.859, 0.859, 0.86 , 0.86 ])

In [44]:
enet_default_cv_2.mean()

0.8611333333333332

In [45]:
enet_to_fit_2 = LogisticRegression(penalty='elasticnet', solver='saga', random_state=101, max_iter=10001)

enet_cv_wflow_2 = Pipeline( steps = [('std_inputs', StandardScaler()), 
                                   ('enet', enet_to_fit_2)] )

enet_search_2 = GridSearchCV(enet_cv_wflow_2, param_grid=enet_grid, cv=my_cv)

enet_search_results_2 = enet_search_2.fit(X_train_2, y_train_2)

enet_search_results_2.best_params_

enet_search_results_2.best_score_

0.8611333333333332

## Case - 3 Treating `X19` as continuous as `X17` and `X06` as categorical

In [46]:
df_3 = pd.read_csv('ppg_churn.csv')

In [47]:
df_3['lump_x17'] = np.where( df_3.X17 > 7, 'Other', df_3.X17.astype('str'))
df_3['X06_cat'] = np.where(df_3['X06'] > 0, 1, 0)

In [48]:
df_3['X06_cat'] = df_3.X06_cat.astype('str')

In [49]:
df_3 = df_3.drop(columns=['X06', 'X17'])

In [50]:
df_3.head()

,state,X02,X03,X04,X05,X07,X08,X09,X10,X11,X12,X13,X14,X15,X16,X18,X19,churn,lump_x17,X06_cat
0,KS,128,AA,Z1,V1,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,2.70,1,no,3,1
1,OH,107,AA,Z1,V1,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3.70,1,no,3,1
2,NJ,137,AA,Z1,V2,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,3.29,0,no,5,0
3,OH,84,BB,Z2,V2,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,1.78,2,no,7,0
4,OK,75,AA,Z2,V2,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,2.73,3,no,3,0


In [51]:
df_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 20 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   state     5000 non-null   object 
 1   X02       5000 non-null   int64  
 2   X03       5000 non-null   object 
 3   X04       5000 non-null   object 
 4   X05       5000 non-null   object 
 5   X07       5000 non-null   float64
 6   X08       5000 non-null   int64  
 7   X09       5000 non-null   float64
 8   X10       5000 non-null   float64
 9   X11       5000 non-null   int64  
 10  X12       5000 non-null   float64
 11  X13       5000 non-null   float64
 12  X14       5000 non-null   int64  
 13  X15       5000 non-null   float64
 14  X16       5000 non-null   float64
 15  X18       5000 non-null   float64
 16  X19       5000 non-null   int64  
 17  churn     5000 non-null   object 
 18  lump_x17  5000 non-null   object 
 19  X06_cat   5000 non-null   object 
dtypes: float64(8), int64(5), objec

In [52]:
xinputs_3 = df_3.select_dtypes('number').copy()
youtput_3 = df_3.loc[:, ['churn']].copy()

X_train_3 = xinputs_3.to_numpy()
y_train_3 = youtput_3.churn.to_numpy().ravel()

In [53]:
X_train_3.shape

(5000, 13)

In [54]:
enet_default_3 = LogisticRegression(penalty='elasticnet', solver='saga', random_state=101, max_iter=10001,
                                  C=1.0, l1_ratio=0.5)

default_enet_wflow_3 = Pipeline( steps=[('std_inputs', StandardScaler()), 
                                      ('enet', enet_default_3)] )

enet_default_cv_3 = cross_val_score(default_enet_wflow_3, X_train_3, y_train_3, cv=my_cv)

enet_default_cv_3

enet_default_cv_3.mean()

0.8584666666666667

In [55]:
enet_to_fit_3 = LogisticRegression(penalty='elasticnet', solver='saga', random_state=101, max_iter=10001)

enet_cv_wflow_3 = Pipeline( steps = [('std_inputs', StandardScaler()), 
                                   ('enet', enet_to_fit_3)] )

enet_search_3 = GridSearchCV(enet_cv_wflow_3, param_grid=enet_grid, cv=my_cv)

enet_search_results_3 = enet_search_3.fit(X_train_3, y_train_3)

enet_search_results_3.best_params_

enet_search_results_3.best_score_

0.8600666666666668

## Case - 4 Treating `X19` and `X17` as continuous and `X06` as categorical

In [56]:
df_4 = pd.read_csv('ppg_churn.csv')

In [57]:
df_4['X06_cat'] = np.where(df_4['X06'] > 0, 1, 0)

In [58]:
df_4['X06_cat'] = df_4.X06_cat.astype('str')

In [59]:
df_4 = df_4.drop(columns=['X06'])

In [60]:
xinputs_4 = df_4.select_dtypes('number').copy()
youtput_4 = df_4.loc[:, ['churn']].copy()

X_train_4 = xinputs_4.to_numpy()
y_train_4 = youtput_4.churn.to_numpy().ravel()

In [61]:
X_train_4.shape

(5000, 14)

In [62]:
enet_default_4 = LogisticRegression(penalty='elasticnet', solver='saga', random_state=101, max_iter=10001,
                                  C=1.0, l1_ratio=0.5)

default_enet_wflow_4 = Pipeline( steps=[('std_inputs', StandardScaler()), 
                                      ('enet', enet_default_4)] )

enet_default_cv_4 = cross_val_score(default_enet_wflow_4, X_train_4, y_train_4, cv=my_cv)

enet_default_cv_4

enet_default_cv_4.mean()

0.8584666666666667

In [63]:
enet_to_fit_4 = LogisticRegression(penalty='elasticnet', solver='saga', random_state=101, max_iter=10001)

enet_cv_wflow_4 = Pipeline( steps = [('std_inputs', StandardScaler()), 
                                   ('enet', enet_to_fit_4)] )

enet_search_4 = GridSearchCV(enet_cv_wflow_4, param_grid=enet_grid, cv=my_cv)

enet_search_results_4 = enet_search_4.fit(X_train_4, y_train_4)

enet_search_results_4.best_params_

enet_search_results_4.best_score_

0.8603999999999999

## Case - 5 Treating `X17` and `X19` as categorical and `X06` as continuous

In [64]:
df_5 = pd.read_csv('ppg_churn.csv')

In [65]:
df_5['lump_x17'] = np.where( df_5.X17 > 7, 'Other', df_5.X17.astype('str'))
df_5['lump_x19'] = np.where( df_5.X19 > 3, 'Other', df_5.X19.astype('str'))

In [66]:
df_5['X06_trans'] = np.log(df_5.X06 + 0.001)

In [67]:
df_5 = df_5.drop(columns=['X06', 'X17', 'X19'])

In [68]:
df_5.shape

(5000, 20)

In [69]:
df_5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 20 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   state      5000 non-null   object 
 1   X02        5000 non-null   int64  
 2   X03        5000 non-null   object 
 3   X04        5000 non-null   object 
 4   X05        5000 non-null   object 
 5   X07        5000 non-null   float64
 6   X08        5000 non-null   int64  
 7   X09        5000 non-null   float64
 8   X10        5000 non-null   float64
 9   X11        5000 non-null   int64  
 10  X12        5000 non-null   float64
 11  X13        5000 non-null   float64
 12  X14        5000 non-null   int64  
 13  X15        5000 non-null   float64
 14  X16        5000 non-null   float64
 15  X18        5000 non-null   float64
 16  churn      5000 non-null   object 
 17  lump_x17   5000 non-null   object 
 18  lump_x19   5000 non-null   object 
 19  X06_trans  5000 non-null   float64
dtypes: float

In [70]:
xinputs_5 = df_5.select_dtypes('number').copy()
youtput_5 = df_5.loc[:, ['churn']].copy()

X_train_5 = xinputs_5.to_numpy()
y_train_5 = youtput_5.churn.to_numpy().ravel()

In [71]:
X_train_5.shape

(5000, 13)

In [72]:
enet_default_5 = LogisticRegression(penalty='elasticnet', solver='saga', random_state=101, max_iter=10001,
                                  C=1.0, l1_ratio=0.5)

default_enet_wflow_5 = Pipeline( steps=[('std_inputs', StandardScaler()), 
                                      ('enet', enet_default_5)] )

enet_default_cv_5 = cross_val_score(default_enet_wflow_5, X_train_5, y_train_5, cv=my_cv)

enet_default_cv_5

enet_default_cv_5.mean()

0.8654666666666666

In [73]:
enet_to_fit_5 = LogisticRegression(penalty='elasticnet', solver='saga', random_state=101, max_iter=10001)

enet_cv_wflow_5 = Pipeline( steps = [('std_inputs', StandardScaler()), 
                                   ('enet', enet_to_fit_5)] )

enet_search_5 = GridSearchCV(enet_cv_wflow_5, param_grid=enet_grid, cv=my_cv)

enet_search_results_5 = enet_search_5.fit(X_train_5, y_train_5)

enet_search_results_5.best_params_

enet_search_results_5.best_score_

0.8656666666666666

## Case - 6 Treating `X19` as categorical and `X17` and  `X06` as continuous

In [74]:
df_6 = pd.read_csv('ppg_churn.csv')

In [75]:
df_6['lump_x19'] = np.where( df_6.X19 > 3, 'Other', df_6.X19.astype('str'))

In [76]:
df_6['X06_trans'] = np.log(df_6.X06 + 0.001)

In [77]:
df_6 = df_6.drop(columns=['X06', 'X19'])

In [78]:
xinputs_6 = df_6.select_dtypes('number').copy()
youtput_6 = df_6.loc[:, ['churn']].copy()

X_train_6 = xinputs_6.to_numpy()
y_train_6 = youtput_6.churn.to_numpy().ravel()

In [79]:
X_train_6.shape

(5000, 14)

In [80]:
enet_default_6 = LogisticRegression(penalty='elasticnet', solver='saga', random_state=101, max_iter=10001,
                                  C=1.0, l1_ratio=0.5)

default_enet_wflow_6 = Pipeline( steps=[('std_inputs', StandardScaler()), 
                                      ('enet', enet_default_6)] )

enet_default_cv_6 = cross_val_score(default_enet_wflow_6, X_train_6, y_train_6, cv=my_cv)

enet_default_cv_6

enet_default_cv_6.mean()

0.8661333333333333

In [81]:
enet_to_fit_6 = LogisticRegression(penalty='elasticnet', solver='saga', random_state=101, max_iter=10001)

enet_cv_wflow_6 = Pipeline( steps = [('std_inputs', StandardScaler()), 
                                   ('enet', enet_to_fit_6)] )

enet_search_6 = GridSearchCV(enet_cv_wflow_6, param_grid=enet_grid, cv=my_cv)

enet_search_results_6 = enet_search_6.fit(X_train_6, y_train_6)

enet_search_results_6.best_params_

enet_search_results_6.best_score_

0.8662666666666667

## Case - 7 Treating `X17` as categorical and `X19` and  `X06` as continuous

In [82]:
df_7 = pd.read_csv('ppg_churn.csv')

In [83]:
df_7['lump_x17'] = np.where( df_7.X17 > 7, 'Other', df_7.X17.astype('str'))

In [84]:
df_7['X06_trans'] = np.log(df_7.X06 + 0.001)

In [85]:
df_7 = df_7.drop(columns=['X06', 'X17'])

In [86]:
xinputs_7 = df_7.select_dtypes('number').copy()
youtput_7 = df_7.loc[:, ['churn']].copy()

X_train_7 = xinputs_7.to_numpy()
y_train_7 = youtput_7.churn.to_numpy().ravel()

In [87]:
X_train_7.shape

(5000, 14)

In [88]:
enet_default_7 = LogisticRegression(penalty='elasticnet', solver='saga', random_state=101, max_iter=10001,
                                  C=1.0, l1_ratio=0.5)

default_enet_wflow_7 = Pipeline( steps=[('std_inputs', StandardScaler()), 
                                      ('enet', enet_default_7)] )

enet_default_cv_7 = cross_val_score(default_enet_wflow_7, X_train_7, y_train_7, cv=my_cv)

enet_default_cv_7

enet_default_cv_7.mean()

0.8632666666666667

In [89]:
enet_to_fit_7 = LogisticRegression(penalty='elasticnet', solver='saga', random_state=101, max_iter=10001)

enet_cv_wflow_7 = Pipeline( steps = [('std_inputs', StandardScaler()), 
                                   ('enet', enet_to_fit_7)] )

enet_search_7 = GridSearchCV(enet_cv_wflow_7, param_grid=enet_grid, cv=my_cv)

enet_search_results_7 = enet_search_7.fit(X_train_7, y_train_7)

enet_search_results_7.best_params_

enet_search_results_7.best_score_

KeyboardInterrupt: 

## Case - 8 Treating `X17`, `X19` and  `X06` as continuous

In [90]:
df_8 = pd.read_csv('ppg_churn.csv')

In [91]:
df_8['X06_trans'] = np.log(df_8.X06 + 0.001)

In [92]:
df_8 = df_8.drop(columns=['X06'])

In [93]:
xinputs_8 = df_8.select_dtypes('number').copy()
youtput_8 = df_8.loc[:, ['churn']].copy()

X_train_8 = xinputs_8.to_numpy()
y_train_8 = youtput_8.churn.to_numpy().ravel()

In [94]:
X_train_8.shape

(5000, 15)

In [ ]:
enet_default_8 = LogisticRegression(penalty='elasticnet', solver='saga', random_state=101, max_iter=10001,
                                  C=1.0, l1_ratio=0.5)

default_enet_wflow_8 = Pipeline( steps=[('std_inputs', StandardScaler()), 
                                      ('enet', enet_default_8)] )

enet_default_cv_8 = cross_val_score(default_enet_wflow_8, X_train_8, y_train_8, cv=my_cv)

enet_default_cv_8

enet_default_cv_8.mean()

In [ ]:
enet_to_fit_8 = LogisticRegression(penalty='elasticnet', solver='saga', random_state=101, max_iter=10001)

enet_cv_wflow_8 = Pipeline( steps = [('std_inputs', StandardScaler()), 
                                   ('enet', enet_to_fit_8)] )

enet_search_8 = GridSearchCV(enet_cv_wflow_8, param_grid=enet_grid, cv=my_cv)

enet_search_results_8 = enet_search_8.fit(X_train_8, y_train_8)

enet_search_results_8.best_params_

enet_search_results_8.best_score_